In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
#import sys.path.append("/home/eze/Escritorio/Exactas/04_métodosNuméricos/07_TP/del_git")
import moduloALC
import alc
from src.alc import matrizConfusion

In [2]:
Xt, Yt, Xv, Yv = alc.cargarDataset("./cats_and_dogs")

### Descomposiciones aparte (después)

In [48]:
#Cholesky

In [3]:
%prun
# W = alc.algoritmo1(Xt[:,900:1100], Yt[:,900:1100])
W = alc.algoritmo1(Xt, Yt)
# W = alc.algoritmo3(Xt, Yt)


 Tamaño A:  (1536, 2000)
Tamaño A:  (1536, 1536)
Tamaño A:  (1536, 1536)


In [4]:
M = W@Xv #[:, 400:600]
M

array([[ 0.41073653,  1.02146944,  2.26842636, ..., -0.02216769,
        -0.37289573, -0.40202675],
       [ 0.57477739,  0.29731716, -1.0880363 , ...,  1.0876697 ,
         1.37626582,  0.95050597]], shape=(2, 1000))

In [5]:
Yv_cortado = Yv #[:, 400:600]
Yv_cortado

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]], shape=(2, 1000))

In [6]:
confusion = matrizConfusion(M, Yv_cortado)

In [7]:
confusion

array([[334., 166.],
       [150., 350.]])